In [25]:
import pandas as pd
import numpy as np
import sklearn.metrics as metrics
import pickle
from glob import glob
import datetime
import requests
import time
from bs4 import BeautifulSoup as bs
import re

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

import os
from time import sleep
from joblib import dump, load

# Project Gutenberg

In [ ]:
# Content was scraped using a wget provided by the website

In [ ]:
files = glob('Gutenberg/txt/*.txt')

book_df = pd.DataFrame(columns=['text', 'info', 'date', 'target'])

In [ ]:
def collect_book_info(files):
    book_info = []
    for file in files:
        open_file = open(file, 'r', encoding= "ISO-8859-1")
        read_file = open_file.read()[:300]
        read_file = read_file.replace(' ', '').split()
        book_info.append(read_file)

    return book_info


book_information = collect_book_info(files)

In [ ]:
def collect_book_text(files):
    book_text = []
    for file in files:
        open_file = open(file, 'r', encoding= "ISO-8859-1")
        read_file = open_file.read()[1500:2300].splitlines()
        book_text.append(read_file)

    return book_text

book_text = collect_book_text(files)

In [ ]:
# Adding information to dataframe
book_df['info'] = book_information
book_df['text'] = book_text

In [ ]:
#Extracting Date Information
def create_date(book_info):
    date_info = []
    for info in book_information:
        info = str(info)
        info = re.findall(r'(\d{4})', info)
        date_info.append(info)

    return date_info

book_dates = create_date(book_information)
book_df['date'] = book_dates

book_df = book_df.replace('[]', np.nan)

In [ ]:
book_df.head()

In [ ]:
book_df['date'] = book_df['date'].astype(str)

In [ ]:
book_df['date'] = book_df['date'].str.strip('[]')
book_df['date'] = book_df['date'].str.strip("''")

In [ ]:
mask = (book_df['date'].str.len() > 4)

len(mask)
book_df[mask] = book_df.loc[mask].date = 'NaN'

# Stylist Magazine Article

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import re


url = 'http://stylist.co.uk/books/the-best-100-closing-lines-from-books/123681'
res = requests.get(url)

soup = bs(res.content, 'lxml')

In [ ]:
soup

In [ ]:
books = []
for link in soup.find_all('h3', {'class': 'css-r1u8am'}):
    ta_dict = {}
    ta_dict['title'] = link.text
    books.append(ta_dict)

In [ ]:
quotes = []
for link in soup.find_all('div', {'class': 'css-dbbd7o'}):
    tr_dict = {}
    tr_dict['quote'] = link.text
    quotes.append(tr_dict)

In [ ]:
book_info = pd.DataFrame(books, quotes)
book_info

In [ ]:
book_info = book_info.reset_index()

In [ ]:
book_info['index'][2]['quote'].splitlines()[0]

In [ ]:
list_of_quotes = book_info['index'].tolist()

In [ ]:
list_of_quotes[0]['quote'].splitlines()[0]

In [ ]:
def extract_quote(list_of_quotes):
    quotes = []
    num = len(list_of_quotes)
    for i in range(0, num):
        new_quote = list_of_quotes[i]['quote'].splitlines()[0]
        quotes.append(new_quote)
    
    return quotes


In [ ]:
book_info['index'] = extract_quote(list_of_quotes)


In [ ]:
book_info = book_info.rename(columns={'index': 'text'})

In [ ]:
df = book_info[book_info['text'].map(len) > 100]

In [ ]:
df = df.reset_index()

In [ ]:
df = df.drop(columns='index')

In [ ]:
df['target'] = 'NaN'

In [ ]:
df['date'] = 'NaN'

In [ ]:
df.to_csv('quote_scrape.csv', index=False)

# LIterature Network

In [ ]:

url = 'http://www.online-literature.com'
res = requests.get(url)

soup = bs(res.content, 'lxml')

In [ ]:
url = 'http://www.online-literature.com/alcott/'

In [ ]:
soup

In [10]:
link_list  = []
browser = webdriver.Firefox()

browser.get('http://www.online-literature.com')
browser.find_element_by_xpath('.//tr[@class="even"]')
result = result.get_attribute('href')


time.sleep(20) 


link_list.append(result)

browser.quit()

AttributeError: 'NoneType' object has no attribute 'get_attribute'

# Reddit

In [ ]:
# url = 'https://www.reddit.com/r/books/comments/1mqfzt/what_is_the_most_powerful_chapter_paragraph_or/.json'
# url = 'https://thoughtcatalog.com/charlie-shaw/2013/09/34-profound-excerpts-from-classic-literature-that-will-change-your-day/.json'
url = 'https://www.reddit.com/r/books/comments/35wv34/whats_the_most_beautiful_paragraph_or_sentence/.json'

In [ ]:
headers = {'User-Agent': 'My User Agent 1.0'}

In [ ]:
def fetch_page(url, after=''):
    params = {'after': after}
    response = requests.get(url, headers=headers, params=params)
    return response.json()[1]['data']['children']

In [ ]:
def parse_post(post):
    keep = ['subreddit', 'title', 'body', 'name'] 
    return {k:v for k, v in post['data'].items() if k in keep}

In [ ]:
def parse_page(page):
    after = ''
    posts = []
    for post in page:
        post = parse_post(post)
        after = post['name']
        posts.append(post)
    return posts, after

In [ ]:
def fetch_subreddit(subreddit, pages=4):
    url = subreddit
    after = ''
    all_posts = []
    for i in range(pages):
        print(f'Fetching Page {i + 1}')
        page = fetch_page(url, after)
        posts, after = parse_page(page)
        all_posts.extend(posts)
        time.sleep(5)
    return all_posts

In [ ]:
posts = fetch_subreddit(url, pages=50)


In [ ]:
reddit_df = pd.DataFrame(posts)

In [ ]:
reddit_df = reddit_df.dropna()

In [ ]:
reddit_df = reddit_df.drop_duplicates()

In [ ]:
def clean_posts(posts):
    new_posts = []
    for post in posts:
        post = post.replace('\n', '')
        new_posts.append(post)
        
    return new_posts

In [ ]:
text = reddit_df['body'].tolist()

In [ ]:
reddit_df['body'] = clean_posts(text)

In [ ]:
def split_column(df, col):
    new = df[col].str.rsplit('-', n=1, expand=True)
    df['text'] = new[0]
    df['info'] = new[1]
    
    return df
# new = reddit_df['body'].str.rsplit('-', n=1, expand=True)

# reddit_df['text'] = new[0]
# reddit_df['info'] = new[1]
df = split_column(reddit_df, 'body')

In [ ]:
df = df.dropna()

In [ ]:
df.shape

In [ ]:
df2 = pd.read_csv('reddit_data.csv', index_col=0)

In [ ]:
df = df.append(df2)

In [ ]:
df.to_csv('reddit_data.csv')

In [ ]:
# df.to_csv('reddit_df.csv')

# Date Scraping (Selenium) 

In [8]:
df = pd.read_csv('../data/processed/reddit_data.csv', index_col=0)
# browser = webdriver.Firefox()

In [9]:
df = df.drop_duplicates(subset=['text'])

In [10]:
df2 = pd.read_csv('../data/processed/quote_scrape.csv')

In [11]:
df = df.drop(columns=['name', 'subreddit', 'body'])

In [12]:
df2 = df2.drop(columns=['target', 'date'])
df2 = df2.rename(columns={'title': 'info'})

In [18]:
df.shape

(138, 2)

In [14]:
df = df.append(df2)

In [15]:
df = df.reset_index()

In [16]:
df = df.drop(columns=['index'])

In [17]:
df = df.drop_duplicates(subset=['text'])

In [19]:
title_list = df['info'].tolist()

In [20]:
search_queries = []
for item in title_list:
    search_queries.append(f'when was {item} published?')

In [ ]:
#Scrape for Google.com

dates  = []
browser = webdriver.Firefox()
for p in search_queries:
    browser.get('http://www.google.com')
    search = browser.find_element_by_name('q')
    search.send_keys(f'"{p}"')
    search.send_keys(Keys.RETURN) # hit return after you enter search text
    time.sleep(20) # sleep for 5 seconds so you can see the results
    
    try:
        result = browser.find_element_by_xpath('.//div[@class="Z0LcW"]')
        result = result.get_attribute('innerHTML')
    except:
        result = 'NaN'

    dates.append(result)
browser.quit()

In [44]:
def extract_years(text):
    new_list_dates = []
    for t in text:
        date = re.findall(r'(\d{4})', t)
        new_list_dates.append(date)
                          
    return new_list_dates

In [26]:
# Scrape for DuckDuckGo.com

In [22]:
dd_sq = []
for item in title_list:
    dd_sq.append(f'what date was {item} published?')

In [23]:

dd_date = []

browser = webdriver.Firefox()
for p in dd_sq:
    browser.get('http://www.duckduckgo.com')
    search = browser.find_element_by_xpath(".//input[@id='search_form_input_homepage']")
    search.send_keys(f'{p}')
    search.send_keys(Keys.RETURN) # hit return after you enter search text
    time.sleep(10) # sleep for 5 seconds so you can see the results
    
    try:
        result = browser.find_element_by_xpath('.//span[@class="js-about-item-abstr"]')
        result = result.get_attribute('innerHTML')
    except:
        result = 'NaN'

    dd_date.append(result)

    
browser.quit()



NoSuchWindowException: Message: Browsing context has been discarded


In [47]:
dd_date
df['duck_dates'] = extract_years(dd_date)

In [50]:
#Scrape for Ask.com

ask_dates = []

browser = webdriver.Firefox()
for p in dd_sq:
    browser.get('http://www.ask.com')
    search = browser.find_element_by_name('q')
    search.send_keys(f'{p}')
    search.send_keys(Keys.RETURN) # hit return after you enter search text
    time.sleep(10) # sleep for 5 seconds so you can see the results
    
    try:
        result = browser.find_element_by_xpath('.//p[@class="PartialSearchResults-item-abstract"]')
        result = result.get_attribute('innerHTML')
    except:
        result = 'NaN'

    ask_dates.append(result)
browser.quit()


In [51]:
df['ask_dates'] = extract_years(ask_dates)

In [52]:
df

,text,info,duck_dates,ask_dates
0,"Really, the whole paragraph is good, but in pa...",A Farewell to Arms,[1929],[1929]
1,"""I looked at the stars, and considered how awf...","Dickens, Great Expectations",[],"[1860, 1861]"
2,"""As the days went by, the evolution of like in...",Jack London,[],[2019]
3,"""Where else? I belong to a lost generation and...","Umberto Eco, Foucault's Pendulum",[1988],"[1988, 1988, 1989]"
4,&gt;Have you ever been in love? Horrible isn't...,"The Sandman, Neil Gaiman",[],[]
5,"Some men are born mediocre, some men achieve m...",Catch 22,"[1953, 1961]","[1953, 1961]"
6,Down there are people who will follow any drag...,The Patrician Vetinari,[],[]
7,This sentence has five words. Here are five mo...,Gary Provost,[],"[1944, 1995]"
8,e live in time - it holds us and molds us - bu...,Sense of an Ending by Julian Barnes,"[2011, 2011]","[2011, 2011]"
9,"Out of the little grove, away from the baffled...","The Amber Spyglass, Phillip Paulman","[2000, 2001]","[2000, 5424, 5587, 0599]"


In [53]:
df.to_csv('../data/processed/reddit_df_with_scraped_dates.csv')